In [1]:
from helical import HyenaDNA, HyenaDNAConfig, HyenaDNAFineTuningModel
from datasets import load_dataset, get_dataset_config_names
import torch
import logging, warnings
import numpy as np

logging.getLogger().setLevel(logging.ERROR)

warnings.filterwarnings("ignore")

2024-09-27 16:48:08.850214: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-27 16:48:08.858823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 16:48:08.868281: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 16:48:08.871122: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 16:48:08.879094: I tensorflow/core/platform/cpu_feature_guar

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
label = "promoter_tata"

configs = get_dataset_config_names("InstaDeepAI/nucleotide_transformer_downstream_tasks_revised", trust_remote_code=True)
dataset = load_dataset("InstaDeepAI/nucleotide_transformer_downstream_tasks_revised", label, trust_remote_code=True)

In [9]:
hyena_config = HyenaDNAConfig(model_name="hyenadna-tiny-1k-seqlen-d256", batch_size=5, device=device)
hyena = HyenaDNA(configurer=hyena_config)

In [10]:
train_dataset = hyena.process_data(dataset["train"]["sequence"])
test_dataset = hyena.process_data(dataset["test"]["sequence"])

Processing sequences: 100%|██████████| 212/212 [00:00<00:00, 10311.51it/s]


In [11]:
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]

In [12]:
len(np.unique(train_labels))

2

In [13]:
hyena_fine_tune = HyenaDNAFineTuningModel(hyena, fine_tuning_head="classification", output_size=len(np.unique(train_labels)))

In [17]:
hyena_fine_tune.train(train_input_data=train_dataset, train_labels=train_labels, validation_input_data=test_dataset, validation_labels=test_labels, optimizer_params={"lr": 0.00001},epochs=5)

Fine-Tuning Validation: 100%|██████████| 43/43 [00:00<00:00, 666.22it/s, accuracy=0.898]
